<div style="padding:30px; color: white; background-color: #0071CD">
<center>
<img src="img/logoub.jpeg"></img>
<p>
<h1>Algorítimica Avançada</h1>
<h2>Exámen práctico II - NQueens </h2>
</center>
</p>
</div>

<div class="alert alert-danger">
<center>
  <h1>ATENCIÓN!</h1>
  
  Para realizar la entrega, hay que subir únicamente este fichero renombrado como: __*grupo*\_*apellidos*\_*nombre*_EP2.ipynb__ donde el grupo ha de ser uno de los cuatro de prácticas (A, B, C o F).
</center>
</div>

<div class="alert alert-info">
<center>
  <h1>Introducción</h1>
</center>
</div>

El objetivo de este examen es solucionar una variante del problema de las N-Reinas que visteis en clase de teoría. En este caso, dado un tablero de $NxN$ buscamos situar el mínimo número de reinas necesarias para que todas las casillas del tablero se encuentren amenazadas por al menos una reina. Recordad que una reina puede atacar en cualquier dirección (horizontal, diagonal y vertical) y a cualquier distancia. En la imagen podéis ver todas las casillas que estarían amenazadas por una reina (en verde) en un tablero de 8x8:
<img src='img/quenmove.png' width=150></IMG>
Una casilla en la que hay una reina la consideraremos también como bajo amenaza. Una posible solución en un tablero de 4x4 con dos reinas se muestra en la siguiente imagen:
<img src='img/quensolve.png' width=150></IMG>

Para simplificar el problema se os ofrece la clase `Queens` que contiene las utilidades necesarias para trabajar con el tablero. Para crear un nuevo tablero de dimensión N llamamos simplemente `board = Queens(N)`. Para añadir una reina en una posición determinada $pos = (p_0,p_1)$, podemos emplear la función `new_board = board.add_queen(pos)`. Cada vez que añadimos una reina, la función nos devuelve __una nueva instancia__ de la clase Queens modificada con la nueva reina en la posición deseada.

También tenemos varios atributos:
<ul>
    <li> <mark>board.free</mark> que nos devolverá una lista con las posiciones que no son amenazadas por ninguna reina.</li>
    <li> <mark>board.empty</mark> que nos devolverá la lista de posiciones en las que no hay ninguna reina. </li> 
    <li> <mark>board.occupied</mark> que nos devolverá la lista de posiciones en las que hay reinas. </li>
<ul>

In [3]:
from queens import Queens

print("Crea un tablero de 3x3")
b1 = Queens(n=3)
print(b1)
print("Pon una reina en (1,0):")
b2 = b1.add_queen((1,0)) # Nótese que b1 != b2
print(b2)
print("Las posiciones que todavía no están amenazadas son:")
print(b2.free)
print("Las posiciones con reians son:")
print(b2.occupied)
print("Las posiciones sin reinas son:")
print(b2.empty)

Crea un tablero de 3x3
+---+---+---+
|   |   |   |
+---+---+---+
|   |   |   |
+---+---+---+
|   |   |   |
+---+---+---+

Pon una reina en (1,0):
+---+---+---+
|   |   |   |
+---+---+---+
| Q |   |   |
+---+---+---+
|   |   |   |
+---+---+---+

Las posiciones que todavía no están amenazadas son:
[(0, 2), (2, 2)]
Las posiciones con reians son:
[(1, 0)]
Las posiciones sin reinas son:
[(0, 0), (0, 1), (0, 2), (1, 1), (1, 2), (2, 0), (2, 1), (2, 2)]


Además tenemos la función `board.is_all_under_attack()` que nos indicará si ya hemos acabado (todas las casillas están amenazadas) `True` o todavía no `False`.

In [4]:
print("El estado de resolución b2 es: %s" % str(b2.is_all_under_attack()))
print(b2)
b3 = b1.add_queen((1,1))
print("El estado de resolución b3 es: %s" % str(b3.is_all_under_attack()))
print(b3)


El estado de resolución b2 es: False
+---+---+---+
|   |   |   |
+---+---+---+
| Q |   |   |
+---+---+---+
|   |   |   |
+---+---+---+

El estado de resolución b3 es: True
+---+---+---+
|   |   |   |
+---+---+---+
|   | Q |   |
+---+---+---+
|   |   |   |
+---+---+---+



<div class="alert alert-info">
<center>
  <h1>Examen</h1>
</center>


<div class="alert alert-success" style="width:90%; margin:0 auto;">

  <p>
  Se pide implementar una función iterativa que, mediante <b>ramificación y poda</b>, resuelva el problema planteado, es decir, que con el mínimo número de reinas consiga que todas las casillas del tablero estén amenazadas por al menos una de ellas. Se valorará principalmente el uso de un algoritmo que siga la estructura de ramificación y poda, no tanto lo eficientes que resulten las cotas a la hora de podar. 
    
   A la hora de probar vuestros algoritmos es recomendable empezar con tamaños de N pequeños (hasta 6). El número de reinas esperadas para tableros de 1 a 8 son: 1, 1, 1, 2, 3, 3, 4 y 5.
  
  Solo se corregirá el código contenido en la función con cabecera `attacking` (de la cual no podéis cambiar la definición) y todo el que ésta llame. Es decir, si creáis funciones o clases auxiliares, solo las corregiremos si se emplean dentro de esta función. Además, no puede existir más de una versión de esta función en el notebook (por ejemplo, en celdas distintas).
  </p>

</div>

In [39]:
"""
    Debéis implementar vuestra solución con la cabecera attacking, la cual tendrá como entradas:
        queens (Queens) --> Tablero sin reinas.
    Y devolverá:
        queens (Queens) --> Tablero con el mínimo número de reinas situadas de 
                            manera tal que todo el tablero esté amenazado.
        expanded (int)  --> Número de nodos de los cuales producimos hijos posibles.
"""
from queens import Queens
import heapq as hq

def attacking(queens):
    
    #Create a upper bound
    best_num_queens = float('inf')
    best_board = queens
    
    #Create a queue to store unexplored states
    #We'll use (current_num_queens + heuristic, curr_num_queens, board)
    unvisited = [(heuristic(queens), 0, 0, queens)]
    
    #save all existent states in order to avoid loops
    existent_states = set([queens])
    
    expanded = 0
    
    while len(unvisited):
        
        #get potentialy best item, i.e. first element in the heapq
        curr_bound, curr_num_queens, curr_level, curr_board = hq.heappop(unvisited)
        
        #check if the bound still potentialy better
        if curr_bound < best_num_queens:
            
            #check if its the solution
            if curr_board.is_all_under_attack():
                best_num_queens = curr_num_queens
                best_board = curr_board
            
            #Iterate throgh all empty possitions
            for new_pos_queen  in curr_board.empty:
                #put a queen there
                new_board = curr_board.add_queen(new_pos_queen)
                new_num_queens = curr_num_queens + 1
                new_bound = new_num_queens + heuristic(new_board)
                
                #print(new_bound)
                
                #if this new board is potentaly better than the
                #current best bound and dosent exist yet
                if new_bound < best_num_queens and new_board not in existent_states:
                    hq.heappush(unvisited, (new_bound, new_num_queens, len(existent_states), new_board))
                    existent_states.add(new_board)
            
            expanded +=1
                
    return best_board,expanded # El tablero solucionado

def heuristic(queens):
    
    A = set()
    
    for x in queens.free:
        A.add(x[0])
        A.add(x[1])
        
    return len(A)/2

def heuristic(queens):
    
    A = set()
    B = set()
    
    for x in queens.free:
        A.add(x[0])
        B.add(x[1])
        
    return max(len(A),len(B))/2

In [40]:
c = Queens(8)
board, expanded = attacking(c)
print(board)
print(expanded)
print(board.is_all_under_attack())

+---+---+---+---+---+---+---+---+
| Q |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   | Q |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   | Q |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   | Q |   |   |   |   |   |   |
+---+---+---+---+---+---+---+---+
|   |   |   |   |   | Q |   |   |
+---+---+---+---+---+---+---+---+

2930
True


<div class="alert alert-info">
<center>
  <h1>Preguntas</h1>
</center>
</div>

### 1. Como has implementado el algoritmo?

Empezando con el tablero vacio, hago una cola de prioridad, i elijo como siguentes estado aquellos en los que puedo poner una reinda en una cleda vacia, y solo ponemos un nuevo eleemnto en la cola si cumple que su cota es menor que la mejor cota que tenemos y adema ese estado no ha sido visitado anteriormente.

### 2. Que criterios utilizas para hacer la ramificación y la poda?

Para la funcion heuristica miro que columnas y filas tienen almenos una cassilla sin ser atacada, entonces como cada reina puede atacar una fila y una columna a la vez miro n el numero de indices distintos, contando filas y columnas, en los que tenemos estas posiciones y esto nos da que tenemos un tablero de nxn, que se puede resolver con un maximo de n/2 reinas.

Una heuristica que creo que es mejor, pero tarda mas dado que descarta menos casos, es la que en vez de tener encuenta los indices de fila y columnas a la vez lo hacemos por separado y entonces con el maximo entre n/2, m/2 (donde n es el numero de filas con una casilla no atacada y m el numero de columnas) de reinas resolvemos el problema.

### 3. Realiza un análisis de complejidad del algoritmo

En el peor de los casos el primer bucle puede visitar todos los possibles estados con n-1 reinas en el tablero, muchos menos realmente, i en cada estado se hace un bucle que como mucho se recorre n² veces. Por lo tanto la complegidad sera O(n!).

<div class="alert alert-info">
<center>
  <h1>Entrega</h1>
</center>
<p>
Se habilitará una tarea en el Campus Virtual pasada media hora desde el comienzo del examen. Una vez terminéis, subid el examen (<b>sólo el notebook</b>) a la tarea "Entrega segundo parcial de pácticas" con vuestro nombre y el grupo de pácticas al que asistís:
</p>
<p>
```
[grupo_practicas]_[apellido]_[nombre]_AA_EP2.ipynb
```

</p>

Donde [grupo_practicas] es uno de entre: A, B, C o F.
<p>
    Por ejemplo, para un alumno llamado <i>Nombre Genérico</i> perteneciente al <i>grupo Z</i> el nombre del archivo debería ser:
</p>
<p>
```
Z_Generico_Nombre_AA_EXP.ipynb
```

Y <b>NO</b>:
<ul>
<li>Z00_Generico_Nombre_AA_EXP.zip</li>
    
<li>ZX_Generico_Nombre_AA_EXP.ipynb</li>

<li>Ex1_Generico_Nombre_AA_EXP.rar</li>

<li>AA_EXP.ipynb</li>
<li> ... cualquier otra cosa</li>
</ul>
</p>
<b>Nota</b>: Recordad la importancia de de mantener el formato facilitado. Nombre de la función, entrada/s y salida/s.
</div>